In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# === CONFIG ===
BASE_URL = "https://quotes.toscrape.com"
HEADERS = {"User-Agent": "Mozilla/5.0 (Educational Scraper - Mukono)"}
OUTPUT_FILE = "quotes_mukono.csv"  # Change 'mukono' only if your surname differs
DELAY = 1.5

# === DATA STORAGE ===
quotes_data = []

# === SCRAPING LOOP ===
page = 1
while True:
    url = f"{BASE_URL}/page/{page}/"
    print(f"Scraping page {page}: {url}")
    
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 404:
            print("Page not found — end of pagination.")
            break
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        break
    
    soup = BeautifulSoup(response.text, "html.parser")
    quote_blocks = soup.select("div.quote")
    
    if not quote_blocks:
        print("No quotes found — likely last page.")
        break
    
    for block in quote_blocks:
        text = block.select_one("span.text").get_text(strip=True).strip('“”')
        author = block.select_one("small.author").get_text(strip=True)
        tags = ", ".join([tag.get_text() for tag in block.select("div.tags a.tag")])
        quote_url = BASE_URL + block.find("a")["href"]
        
        quotes_data.append({
            "Quote": text,
            "Author": author,
            "Tags": tags,
            "URL": quote_url
        })
    
    page += 1
    time.sleep(DELAY)

# === SAVE TO CSV ===
if quotes_data:
    keys = quotes_data[0].keys()
    with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        writer.writerows(quotes_data)
    
    print(f"\nSuccess! {len(quotes_data)} quotes saved to {OUTPUT_FILE}")
else:
    print("No data collected.")